#               Predicting survival from titanic crash

In [174]:
import pandas as pd

In [175]:
df = pd.read_csv("titanic.csv")
df.head()

,PassengerId,Name,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,1,"Braund, Mr. Owen Harris",3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,"Heikkinen, Miss. Laina",3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,female,35.0,1,0,113803,53.1000,C123,S,1
4,5,"Allen, Mr. William Henry",3,male,35.0,0,0,373450,8.0500,NaN,S,0


In [176]:
df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis=1,inplace=True)
df.head()

,Pclass,Sex,Age,Fare,Survived
0,3,male,22.0,7.2500,0
1,1,female,38.0,71.2833,1
2,3,female,26.0,7.9250,1
3,1,female,35.0,53.1000,1
4,3,male,35.0,8.0500,0


In [177]:
inputs = df.drop('Survived',axis=1)
target = df.Survived

In [178]:
inputs.head()

,Pclass,Sex,Age,Fare
0,3,male,22.0,7.2500
1,1,female,38.0,71.2833
2,3,female,26.0,7.9250
3,1,female,35.0,53.1000
4,3,male,35.0,8.0500


In [179]:
target.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [180]:
dummies = pd.get_dummies(inputs.Sex)
dummies.head()

,female,male
0,0,1
1,1,0
2,1,0
3,1,0
4,0,1


In [181]:
inputs = pd.concat([inputs,dummies],axis=1)
inputs.head()

,Pclass,Sex,Age,Fare,female,male
0,3,male,22.0,7.2500,0,1
1,1,female,38.0,71.2833,1,0
2,3,female,26.0,7.9250,1,0
3,1,female,35.0,53.1000,1,0
4,3,male,35.0,8.0500,0,1


# I am dropping sex column as well because of object variable trap theory.

In [182]:
inputs.drop(['Sex'],axis=1,inplace=True)
inputs.head()

,Pclass,Age,Fare,female,male
0,3,22.0,7.2500,0,1
1,1,38.0,71.2833,1,0
2,3,26.0,7.9250,1,0
3,1,35.0,53.1000,1,0
4,3,35.0,8.0500,0,1


In [183]:
#Use Series.isna() function to detect missing values in the given series object.
inputs.columns[inputs.isna().any()]

Index(['Age'], dtype='object')

In [184]:
inputs.Age[:10]

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: Age, dtype: float64

In [185]:
inputs.Age = inputs.Age.fillna(inputs.Age.mean())
inputs.head()

,Pclass,Age,Fare,female,male
0,3,22.0,7.2500,0,1
1,1,38.0,71.2833,1,0
2,3,26.0,7.9250,1,0
3,1,35.0,53.1000,1,0
4,3,35.0,8.0500,0,1


In [186]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(inputs,target,test_size=0.3)

In [187]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

In [188]:
model.fit(X_train,y_train)

GaussianNB()

In [189]:
model.score(X_test,y_test)

0.7873134328358209

In [190]:
X_test[0:10]

,Pclass,Age,Fare,female,male
123,2,32.500000,13.0000,1,0
33,2,66.000000,10.5000,0,1
515,1,47.000000,34.0208,0,1
804,3,27.000000,6.9750,0,1
716,1,38.000000,227.5250,1,0
583,1,36.000000,40.1250,0,1
34,1,28.000000,82.1708,0,1
487,1,58.000000,29.7000,0,1
739,3,29.699118,7.8958,0,1
298,1,29.699118,30.5000,0,1


In [191]:
y_test[0:10]

123    1
33     0
515    0
804    1
716    1
583    0
34     0
487    0
739    0
298    1
Name: Survived, dtype: int64

In [192]:
model.predict(X_test[0:10])

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0])

In [193]:
model.predict_proba(X_test[:10])

array([[3.29412223e-02, 9.67058778e-01],
       [9.60526416e-01, 3.94735839e-02],
       [8.92212547e-01, 1.07787453e-01],
       [9.90380902e-01, 9.61909828e-03],
       [1.40546307e-09, 9.99999999e-01],
       [8.89742762e-01, 1.10257238e-01],
       [6.56675780e-01, 3.43324220e-01],
       [8.69220733e-01, 1.30779267e-01],
       [9.90789573e-01, 9.21042748e-03],
       [8.99033689e-01, 1.00966311e-01]])

# Calculate the score using cross validation

In [194]:
from sklearn.model_selection import cross_val_score
cross_val_score(GaussianNB(),X_train,y_train,cv=7)

array([0.73033708, 0.79775281, 0.7752809 , 0.71910112, 0.79775281,
       0.85393258, 0.78651685])